In [1]:
%env CUDA_LAUNCH_BLOCKING=1

env: CUDA_LAUNCH_BLOCKING=1


In [2]:
%load_ext autoreload
%autoreload complete
# %aimport ...

import sys
import importlib
import warnings

importlib.invalidate_caches()

for mod_name in list(sys.modules):
    if not (str.startswith(mod_name, "robotodo") or str.startswith(mod_name, "tensorspecs")):
        continue
    try:
        importlib.reload(sys.modules[mod_name])
    except Exception as error:
        warnings.warn(f"{error}")

In [3]:
from robotodo.engines.isaac._kernel import Kernel

kernel = Kernel([
    "--/log/file=/tmp/todo-isaacsim.log",
    "--/log/enableStandardStreamOutput=true",
], kit_path="isaacsim.exp.full.kit")
kernel.start_app_loop_soon()

[Warning] [simulation_app] Interactive python shell detected but ISAAC_JUPYTER_KERNEL was not set. Problems with asyncio may occur
[Warning] [simulation_app] Please use Isaac Sim Python 3 kernel instead of the default Python 3 Kernel
Loading user config located at: '/home/sysadmin/lab/robotodo/.conda/lib/python3.11/site-packages/omni/data/Kit/Isaac-Sim Full/5.0/user.config.json'
[Info] [carb] Logging to file: /tmp/todo-isaacsim.log

|---------------------------------------------------------------------------------------------|
| Driver Version: 535.247.01    | Graphics API: Vulkan
|=============================================================================================|
| GPU | Name                             | Active | LDA | GPU Memory | Vendor-ID | LUID       |
|     |                                  |        |     |            | Device-ID | UUID       |
|     |                                  |        |     |            | Bus-ID    |            |
|---------------------------

Checking to see if RMW can be loaded:
failed to get symbol 'rmw_init_options_init' due to Environment variable 'AMENT_PREFIX_PATH' is not set or empty, at /workspace/humble_ws/src/rmw_implementation/src/functions.cpp:171, at /workspace/humble_ws/src/rcl/rcl/src/rcl/init_options.c:75
RMW was not loaded



2025-10-11T07:47:55Z [14,010ms] [Error] [isaacsim.ros2.bridge.impl.extension] ROS2 Bridge startup failed
2025-10-11T07:47:57Z [15,467ms] [Error] [carb.graphics-vulkan.plugin] Could not get NGX parameters block because NGX isn't enabled.
2025-10-11T07:47:57Z [15,467ms] [Error] [carb.graphics-vulkan.plugin] Failed to create NGX context.
2025-10-11T07:47:57Z [15,594ms] [Error] [rtx.optixdenoising73.plugin] Failed to create an Optix adaptor for device 0


Got contact header type: ContactEventType.CONTACT_FOUND
Actor0: /World/Cube
Actor1: /SimpleRoom/table_low_327/table_low
Collider0: /World/Cube
Collider1: /SimpleRoom/table_low_327/table_low
StageId: 9223001
Number of contacts: 4
Contact:
Contact position: (-0.5,0.5,0.0152562)
Contact normal: (0,0,1)
Contact impulse: (0,0,1112.2)
Contact separation: 0.022642076015472412
Contact faceIndex0: 0
Contact faceIndex1: 0
Contact material0: 
Contact material1: 
Contact:
Contact position: (0.5,0.5,0.0152562)
Contact normal: (0,0,1)
Contact impulse: (0,0,674.575)
Contact separation: 0.022642076015472412
Contact faceIndex0: 0
Contact faceIndex1: 0
Contact material0: 
Contact material1: 
Contact:
Contact position: (0.5,-0.5,0.0152562)
Contact normal: (0,0,1)
Contact impulse: (0,0,1119)
Contact separation: 0.022642076015472412
Contact faceIndex0: 0
Contact faceIndex1: 0
Contact material0: 
Contact material1: 
Contact:
Contact position: (-0.5,-0.5,0.0152562)
Contact normal: (0,0,1)
Contact impulse: (0

In [4]:
from robotodo.engines.isaac.scene import Scene

scene = Scene(_kernel=kernel)

In [5]:
list(scene.traverse())

['/',
 '/World',
 '/Environment',
 '/Environment/defaultLight',
 '/Render',
 '/Render/OmniverseKit',
 '/Render/OmniverseKit/HydraTextures',
 '/Render/OmniverseKit/HydraTextures/omni_kit_widget_viewport_ViewportTexture_0',
 '/Render/OmniverseGlobalRenderSettings',
 '/Render/Vars',
 '/Render/Vars/LdrColor',
 '/OmniverseKit_Persp',
 '/OmniverseKit_Front',
 '/OmniverseKit_Top',
 '/OmniverseKit_Right']

In [18]:
from robotodo.engines.isaac.entity import Entity

entity = Entity("/**/*", scene=scene)

In [19]:
entity.collidable = True

In [20]:
entity.collidable

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])

In [ ]:
await anext(entity.on_contact)

CancelledError: 

: 

In [6]:
import omni

import carb

carb.settings.get_settings().get(
    omni.physx.bindings._physx.SETTING_ENABLE_DEFORMABLE_BETA
)


True

In [ ]:
from pxr import Usd, UsdGeom, Vt, Gf

def create_trimesh(stage, path, res):
    triMesh = UsdGeom.Mesh.Define(stage, path)
    step = 1.0 / res
    verts = [(i * step, j * step, 0.0) for j in range(res + 1) for i in range(res + 1)]
    idx = lambda i, j: j * (res + 1) + i
    tris = [(idx(i,j), idx(i+1,j), idx(i+1,j+1)) + (idx(i,j), idx(i+1,j+1), idx(i,j+1))
            for j in range(res) for i in range(res)]
    triMesh.GetPointsAttr().Set(Vt.Vec3fArray(verts))
    triMesh.GetFaceVertexCountsAttr().Set([3] * (2 * res**2))
    triMesh.GetFaceVertexIndicesAttr().Set([i for t in tris for i in t])
    return triMesh


from pxr import Usd, UsdGeom, Vt, Gf, UsdPhysics, PhysxSchema
import omni.usd

stage = omni.usd.get_context().get_stage()
triMesh = create_trimesh(stage, "/World/TriMesh", 6)
prim = triMesh.GetPrim()

# Apply deformable body API and set mass
prim.ApplyAPI("OmniPhysicsDeformableBodyAPI")
if prim.HasAPI("OmniPhysicsDeformableBodyAPI"):
    prim.GetAttribute("omniphysics:mass").Set(0.5)

# Apply volume simulation API and set rest shape properties
prim.ApplyAPI("OmniPhysicsSurfaceDeformableSimAPI")
if prim.HasAPI("OmniPhysicsSurfaceDeformableSimAPI"):
    restShapePointsAttr = prim.GetAttribute("omniphysics:restShapePoints")
    restShapePointsAttr.Set(triMesh.GetPointsAttr().Get())
    restTriVtxIndicesAttr = prim.GetAttribute("omniphysics:restTriVtxIndices")
    # Convert face vertex indices to Gf.Vec3i indices
    triVtxIndices = list(zip(*[iter(triMesh.GetFaceVertexIndicesAttr().Get())]*3))
    restTriVtxIndicesAttr.Set(triVtxIndices)

# Apply collision API
collisionAPI = UsdPhysics.CollisionAPI.Apply(prim)

# Optionally set physx specific APIs and properties

prim.ApplyAPI("PhysxSurfaceDeformableBodyAPI")
if prim.HasAPI("PhysxSurfaceDeformableBodyAPI"):
    # prim.GetAttribute("physxDeformableBody:disableGravity").Set(True)
    prim.GetAttribute("physxDeformableBody:disableGravity").Set(False)
    prim.GetAttribute("physxDeformableBody:selfCollision").Set(True)

physxCollisionAPI = PhysxSchema.PhysxCollisionAPI.Apply(prim)
if physxCollisionAPI:
    physxCollisionAPI.GetContactOffsetAttr().Set(0.02)
    physxCollisionAPI.GetRestOffsetAttr().Set(0.01)


from pxr import Usd, UsdGeom, Sdf, Gf, Tf, Vt, UsdPhysics, PhysxSchema



True

In [8]:
for prim in stage.Traverse():
    c = PhysxSchema.PhysxContactReportAPI.Apply(prim)
    c.CreateThresholdAttr().Set(0)

In [9]:
import omni

from omni.physx.scripts.physicsUtils import *
from pxr import Usd, UsdGeom, Sdf, Gf, Tf, Vt, UsdPhysics, PhysxSchema
from omni.physx import get_physx_interface
from pxr import PhysicsSchemaTools


def _on_contact_report_event(contact_headers, contact_data):
    for contact_header in contact_headers:
        print("Got contact header type: " + str(contact_header.type))
        print("Actor0: " + str(PhysicsSchemaTools.intToSdfPath(contact_header.actor0)))
        print("Actor1: " + str(PhysicsSchemaTools.intToSdfPath(contact_header.actor1)))
        print("Collider0: " + str(PhysicsSchemaTools.intToSdfPath(contact_header.collider0)))
        print("Collider1: " + str(PhysicsSchemaTools.intToSdfPath(contact_header.collider1)))
        print("StageId: " + str(contact_header.stage_id))
        print("Number of contacts: " + str(contact_header.num_contact_data))

        contact_data_offset = contact_header.contact_data_offset
        num_contact_data = contact_header.num_contact_data

        for index in range(contact_data_offset, contact_data_offset + num_contact_data, 1):
            print("Contact:")
            print("Contact position: " + str(contact_data[index].position))
            print("Contact normal: " + str(contact_data[index].normal))
            print("Contact impulse: " + str(contact_data[index].impulse))
            print("Contact separation: " + str(contact_data[index].separation))
            print("Contact faceIndex0: " + str(contact_data[index].face_index0))
            print("Contact faceIndex1: " + str(contact_data[index].face_index1))
            print("Contact material0: " + str(PhysicsSchemaTools.intToSdfPath(contact_data[index].material0)))
            print("Contact material1: " + str(PhysicsSchemaTools.intToSdfPath(contact_data[index].material1)))


iface = omni.physx.get_physx_simulation_interface()
iface.subscribe_contact_report_events(_on_contact_report_event)